<div class="alert alert-warning" role="alert">
  <h3>📋任务</h3> 
    现在大家可以将无风险资产纳入投资组合中，存在无风险资产时，投资者可以通过贷出自有资金减少自己的风险，当然这样会降低期望报酬率；也可以通过借入资金，增加购买风险资产的投入资金，从而提高期望报酬率。这样也就意味着投资者可以按照无风险利率借入或者借出任何数量的资金， 这样就引出的是：<strong>资产市场线</strong>.
</div>

假设投资组合的风险资产$M$和无风险资产资产$F$，设$R_M$为风险资产收益率，风险资产的期望收益$E(R_M)$，风险资产的波动率为：$\sigma_M$，无风险资产的收益率为$R_F$，无风险资产的波动率为0（无风险）。

假设风险资产与无风险资产的投资比例为$y$和$1-y$，那么整个组合的收益率为：$R_p = yE(R_M)+(1-y)R_F = R_F+y[E(R_M)-R_F]$。因此整个组合的预期收益率为：
$$
E(R_p) = R_F+y[E(R_M)-R_F]
$$
整个组合的波动率为：
$$
\sigma_p = y\sigma_M
$$
因此，$y = \frac{\sigma_p}{\sigma_M}$，代入整个组合的预期收益率方程得：$E(R_p) = R_F+[E(R_M)-R_F]\frac{\sigma_p}{\sigma_M}$，我们把$\frac{E(R_M)-R_F}{\sigma_M}$称为夏普比率，夏普比率就是整个组合预期收益率直线的斜率，我们希望斜率越大越好，这样获得的收益就会越大。

$\frac{E\left(R_{M}\right)-R_{F}}{\sigma_{M}}$ 是资本市场线的斜率, 也就是夏普比率 (Sharpe Ratio)。

因此：计算资本市场线的斜率就是求解以下优化问题：
$$
\max _{w_{i}} \frac{E\left(R_{M}\right)-R_{F}}{\sigma_{M}} 
$$
约束为$\sum_{i=1}^{N} w_{i}=1$，其中$w_i$为第$i$种风险资产在风险资产组合中的比例。


In [ ]:
import matplotlib as plt
import numpy as np
import scipy.optimize as sco
import baostock as bs

# 计算对数收益率
R = np.log(data_stock/data_stock.shift(1)) # log(y_t) - log(y_{t-1})
R_mean = R.mean()*252   # 计算股票的平均年化收益率
R_cov = R.cov()*252   # 计算股票的协方差矩阵并且年化处理

In [ ]:
# 求解资本市场线(请在“----------------”处补全代码)
def F(w):
    Rf = 0.02
    w = np.array(w)
    # 请根据风险组合的预期收益率公式补全以下代码
    RM_opt = np.sum(w*R_mean)  # E(R_M)
    # 请根据市场组合的波动率补全以下代码
    VM_opt = np.sqrt(np.dot(w,np.dot(R_cov,w.T))) # sigma_M
    # 请根据夏普比率公式补全以下代码
    SR = (RM_opt-Rf)/VM_opt
    return np.array([RM_opt,VM_opt,SR])

# 最大值等于最小化负的最小值
def SRmin_F(w):
    return -F(w)[2]

# 请根据约束：所有风险组合的投资比例和为1的约束补全以下代码
cons_SR = ({'type':'eq','fun':lambda x:np.sum(x)-1})
bnds = tuple((0,1) for x in range(len(R_mean))) # 每个权重必须要在区间(0,1)，禁止卖空
# 请根据最优化的目标表达式和约束条件补全以下代码
result_SR = sco.max(SRmin_F,len(R_mean)*[1.0/len(R_mean)],method='SLSQP',bounds=bnds,constraints=cons_SR)
Rf = 0.02
slope = -result_SR['fun']   # 资本市场线斜率
Rm = np.sum(R_mean*result_SR['x'])   #计算风险组合的预期收益率
Vm = (Rm-Rf) / slope # #计算风险组合的波动率
print('市场组合的预期收益率',round(Rm,4))
print('市场组合的波动率',round(Vm,4))

In [ ]:
# 资本市场线可视化
Rp_cml = np.linspace(0.02,0.25)   
Vp_cml = (Rp_cml-Rf) / slope

plt.figure(figsize=(8,6))
plt.scatter(Vp_list,Rp_list)
plt.plot(Vp_target,Rp_target,'r-',label=u'有效前沿',lw=2.5)
plt.plot(Vp_cml,Rp_cml,'b--',label=u'资本市场线',lw=2.5)
plt.plot(Vp_vmin,Rp_vmin,'b*',label=u'全局最小波动率',markersize=18)
plt.plot(Vm,Rm,'g*',label=u'市场组合',markersize=14)
plt.xlabel(u'波动率',fontsize=13)
plt.ylabel(u'收益率',fontsize=13,rotation=90)
plt.xticks(fontsize=13)
plt.yticks(fontsize=13)
plt.xlim(0,0.32)
plt.ylim(-0.1,0.25)
plt.title(u'投资组合理论的可视化',fontsize=13)
plt.legend(fontsize=13)
plt.show()


在蓝色的资本市场线上，当$x=0$时，代表购入全部的资产都是无风险资产，绿色的市场组合点表示所有资金都投资风险资产，因此在资本市场线上可以出现以下情况：

| 风险资产占总投资资产的投资比例 | 投资情况                    | 期望报酬率     | 风险     |
| :-------------- | :---------------------- | :-------- | :----- |
| y > 1           | 把自有资金与借入资金都投入风险投资       | 高于市场报酬率   | 高于市场风险 |
| y = 1           | 把自有资金全部投入风险投资           | 等于市场报酬率   | 等于市场风险 |
| y < 1           | 把自有资金部分投资风险资产，部分投资无风险资产 | 低于市场报酬率   | 低于市场风险 |
| y = 0           | 把自有资产全部投资于无风险资产         | 等于无风险资产利率 | 无风险    |

资本市场线实际上指出了有效投资组合风险与预期收益率之间的关系，提供了衡量有效投资组合风险的方法。对于每一个有效投资组合而言，给定其风险的大小，便可根据资本市场线知道其预期收益率的大小。